Configurações iniciais

In [16]:
!pip install --quiet --upgrade tensorflow_federated


In [17]:
import nest_asyncio
nest_asyncio.apply()
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [18]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
import math
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
import keras

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout

from sklearn.model_selection import train_test_split

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()


b'Hello, World!'

In [19]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/creditcard.csv')

In [20]:
y = df.Class.to_numpy()
x = df.drop(columns = ['Class'])
x = x.iloc[:,1:].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size = 0.2,
    shuffle= True,
    )

In [21]:
counts = np.bincount(y_train[:])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(y_train)
    )
)
counts2 = np.bincount(y_test[:])
print(
    "Number of positive samples in test data: {} ({:.2f}% of total)".format(
        counts2[1], 100 * float(counts2[1]) / len(y_test)
    )
)


weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]
class_weight = {0: weight_for_0, 1: weight_for_1}

Number of positive samples in training data: 391 (0.17% of total)
Number of positive samples in test data: 101 (0.18% of total)


In [22]:
mean = np.mean(x_train, axis=0)
x_train -= mean
x_test -= mean
std = np.std(x_train, axis=0)
x_train /= std
x_test /= std


In [23]:
NUM_CLIENTS = 3

xs = np.array_split(x_train, NUM_CLIENTS)
ys = np.array_split(y_train, NUM_CLIENTS)

In [24]:
NUM_ROUNDS = 5
NUM_EPOCHS = 15
BATCH_SIZE = 20
PREFETCH_BUFFER = 10


In [25]:
def model_fn():
  model = Sequential([
    Dense(256, activation="relu", input_shape=(x_train.shape[-1],)),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
  ])
  model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
      keras.metrics.FalseNegatives(name="fn"),
      keras.metrics.FalsePositives(name="fp"),
      keras.metrics.TrueNegatives(name="tn"),
      keras.metrics.TruePositives(name="tp"),
      keras.metrics.Precision(name="precision"),
      keras.metrics.Recall(name="recall")
            ],
    optimizer=keras.optimizers.Adam(1e-2))
  return model

In [26]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

def client_train(model, client_x, client_y):
  counts = np.bincount(client_y[:])
  weight_for_0 = 1.0 / counts[0]
  weight_for_1 = 1.0 / counts[1]
  class_weight = {0: weight_for_0, 1: weight_for_1}

  model.fit(
    client_x,
    client_y,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_split = 0.2,
    verbose=0,
    callbacks=[early_stop],
    class_weight=class_weight
  )

  return model

In [28]:
from operator import add
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)   

tff_val_prec = []
tff_val_loss = []

global_model = model_fn()
for round_num in range(1, NUM_ROUNDS+1): 
  first = True
  for client_x, client_y in zip(xs, ys):
    if first:
      c_model = client_train(global_model, client_x, client_y)
      weights = c_model.get_weights()
      first = False
      global_model.set_weights(weights)
    else:
      c_model = client_train(global_model, client_x, client_y)
      new_weights = c_model.get_weights()
      weights = list( map(add, weights, new_weights) )
  avg_weights = np.divide(weights, NUM_CLIENTS)
  global_model.set_weights(avg_weights)
  # state.model.assign_weights_to(model=eval_model)

  ev_result = global_model.evaluate(x_test, y_test, verbose=0, return_dict=True)
  print(f"Round {round_num} -> Loss : {ev_result['loss']} | Precision : {ev_result['precision']} | TruePositive : {ev_result['tp']} | TrueNegative : {ev_result['tn']} | FalsePositive : {ev_result['fp']} | FalseNegative : {ev_result['fn']}")
  tff_val_prec.append(ev_result['precision'])
  tff_val_loss.append(ev_result['loss'])

metric_collection = {"precision": tff_val_prec,
                     "loss": tff_val_loss}

In [30]:
print(f"Round {round_num} -> Loss : {ev_result['loss']} | Precision : {ev_result['precision']} | TruePositive : {ev_result['tp']} | TrueNegative : {ev_result['tn']} | FalsePositive : {ev_result['fp']} | FalseNegative : {ev_result['fn']}")


Round 5 -> Loss : 0.7232362627983093 | Precision : 0.0017731118714436889 | TruePositive : 101.0 | TrueNegative : 0.0 | FalsePositive : 56861.0 | FalseNegative : 0.0
